In [11]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.0f}'.format)


import f_validate_g4s as ed

from time import time
from datetime import timedelta

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'\nCreated on Wed Feb 19 16:21:00 2020\n\n@author: cmchico\n\nusual imports\n'

In [12]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

'\nCreated on Wed Feb 19 16:24:03 2020\n\n@author: cmchico\n'

Client Scheduler: tcp://127.0.0.1:52358 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [13]:
path = "D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/"

In [18]:
test_dates = pd.date_range(start='01/01/2019',end='12/31/2019')
mapp = pd.read_pickle(path+'Actual/89 TID Br map.pkl')
#211 branches with just one ATM, can be treated as an ATM i/o Branch
mapp[mapp.TID!=mapp.TID_BR].BR_TIDCOUNT.value_counts()

2    256
1    211
3     42
4      8
Name: BR_TIDCOUNT, dtype: int64

In [19]:
reload_df = pd.read_pickle(path+'Actual/80 Replenishment.pkl')
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['Pre-Emptive'] = reload_df['Pre-Emptive'].str.strip().fillna("NORMAL LOADING")
reload_df['date_str'] = reload_df.DATE.astype(str)


reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(str).str.strip().str.replace("","")
select = reload_df['Current Load Amount']==""
reload_df.loc[select,'Current Load Amount'] = "0"
reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(int)

reload_df = reload_df[reload_df.DATE.isin(test_dates)].copy()
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['TID_BR'] = reload_df.TID.map(mapp.set_index('TID').TID_BR.to_dict())

first_reload = reload_df.drop_duplicates(subset=(['TID'])).copy()

reload_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194966 entries, 0 to 194965
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Last Load Amount     194965 non-null  object        
 1   Current Load Amount  194966 non-null  int32         
 2   % Cash Return/Atm    194966 non-null  float64       
 3   Replen               194966 non-null  object        
 4   Pre-Emptive          194966 non-null  object        
 5   TID                  194966 non-null  object        
 6   DATE                 194966 non-null  datetime64[ns]
 7   date_str             194966 non-null  object        
 8   TID_BR               120787 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(6)
memory usage: 12.6+ MB


In [28]:
preempt = reload_df[reload_df['Pre-Emptive'] =='PRE-EMPTIVE DUE TO CF'][['date_str','TID_BR','Pre-Emptive','Current Load Amount']]

In [21]:
df = pd.concat([pd.read_csv(path+'Prophet Forecast/'+i, dtype={'ds':'str','pred_date':'str','tid':'str'}) 
                for i in os.listdir(path+'Prophet Forecast/')],ignore_index=True)
df['pred_date_D'] = pd.to_datetime(df.pred_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2255526 entries, 0 to 2255525
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Unnamed: 0   int64         
 1   ds           object        
 2   y            float64       
 3   yhat         float64       
 4   pred_date    object        
 5   pred_day     object        
 6   se           float64       
 7   tid          object        
 8   pred_date_D  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 154.9+ MB


In [22]:
df.sort_values(['tid','pred_date','ds'], inplace=True)

In [23]:
actual = df[df.pred_day.isin(['day 0','day 1','day 2'])].pivot_table(index=['tid','pred_date','pred_date_D'],columns='pred_day',values='yhat',aggfunc=sum).reset_index()
actual.columns=['tid','pred_date','pred_date_D','yhat0','yhat1','yhat2']
actual[['date_str','y']] = df[df.pred_day=='day 0'][['ds','y']].reset_index(drop=True)

In [24]:
atms = [atm for atm in actual.tid.unique() if atm in mapp[mapp.BR_TIDCOUNT==1].TID_BR.unique()]
len(atms)

393

In [25]:
def par(atm,rel_amt):
    
    date_o = first_reload.loc[first_reload['TID_BR'] == atm].reset_index().DATE[0]
    actual_atm = actual[(actual.tid== atm) & (actual.pred_date_D >= date_o-timedelta(days=1))].reset_index(drop=True).copy()

    g4s=preempt[preempt.TID_BR==atm].set_index('date_str')
    g4s['Current Load Amount'] = rel_amt
    
    if g4s.shape[0]==0: g4s=None
    sched_= ed.sched(actual_atm,rel_amt=rel_amt,pred=pred,r=r,reload_charge=reload_charge,
                      threshold=threshold, hor=hor,g4s=g4s)
    sched_['fix_rel']=rel_amt
    ss=sched_.groupby('cog_preemptive') \
                        [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
                                             'civ_cost','reload_cost']].sum().reset_index()
    ss['tid']=atm
    ss['fix_rel'] = rel_amt
    sched_.to_csv(path+'Prophet Test Thresh 25/_sched_'+atm+'_'+str(rel_amt)[:2]+'.csv')
    ss.to_csv(path+'Prophet Test Perf Thresh 25/_schedperf_'+ atm+'_'+str(rel_amt)[:2]+'.csv')
    
    return sched_,ss

In [29]:
rel_amt=3200000
pred = ['yhat0','yhat1','yhat2']
hor=len(pred)-1
r = 0.05/365
reload_charge=2000
threshold=0.25


# start=time()
# for atm in atms[:1]:
#     actual_atm = actual[actual.tid==atm].reset_index(drop=True)
#     g4s=preempt[preempt.TID==atm].set_index('date_str')
    
#     if g4s.shape[0]==0: g4s=None
#     sched_= ed.sched(actual_atm,rel_amt=rel_amt,pred=pred,r=r,reload_charge=reload_charge,
#                       threshold=threshold, hor=hor,g4s=g4s)
#     ss=sched_.groupby('cog_preemptive') \
#                         [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
#                                              'civ_cost','reload_cost']].sum().reset_index()
#     ss['tid']=atm
    
#     sched_.to_csv(path+'/Prophet Test/_sched_'+atm+'.csv')
#     ss.to_csv(path+'Prophet Test Perf/_schedperf_'+ atm+'.csv')
#     print(atm)
#     print(timedelta(seconds=time()-start))

In [31]:
dbatms = db.from_sequence(atms)
dbmaster = dbatms.map(par,rel_amt=3200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:06:39.649367


In [32]:
dbatms = db.from_sequence(atms)
dbmaster = dbatms.map(par,rel_amt=4200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:06:35.576525


In [33]:
dbatms = db.from_sequence(atms)
dbmaster = dbatms.map(par,rel_amt=5200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:06:41.222555


In [34]:
dbatms = db.from_sequence(atms)
dbmaster = dbatms.map(par,rel_amt=6200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:06:17.943774
